# ResNet-18 Torch example  
This notebook will download a trained ResNet-18 from [pytorch models](https://download.pytorch.org/models/resnet18-f37072fd.pth) and stores it locally for further usage, 
[Inference.ipynb](./Inference.ipynb)

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import requests
from io import BytesIO


In [ ]:
import os

def create_folder(path):
    try:
        os.makedirs(path, exist_ok=True)
        print(f"Folder '{path}' created successfully.")
    except OSError as e:
        print(f"Error creating folder '{path}': {e}")



In [ ]:

# Load pre-trained ResNet-18 model
model = models.resnet18(pretrained=True)
model.eval()
# Append a softmax layer to the model
#model = nn.Sequential(model, nn.Softmax(dim=0)) ##!not working!!
folder_path = 'zoo'
create_folder(folder_path)
torch.save(model.state_dict(), 'zoo/resnet18_model.pth')


In [ ]:
from torchsummary import summary
summary(model, (3, 224, 224))

In [ ]:

folder_path = 'onnx'
create_folder(folder_path)
# Example input to the model
example_input = torch.randn(1, 3, 224, 224)

# Export the model to ONNX format
onnx_path = "onnx/resnet18.onnx"
torch.onnx.export(model, example_input, onnx_path, verbose=True)

print(f"Model exported to {onnx_path}")


In [ ]:

# Perform inference on an example image
image_url = 'https://www.hartz.com/wp-content/uploads/2022/04/small-dog-owners-1.jpg'  # Provide an image URL
response = requests.get(image_url,verify=False)
img = Image.open(BytesIO(response.content))


In [ ]:
# Preprocess the image
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

input_data = preprocess(img)
input_data = input_data.unsqueeze(0)  


In [ ]:

# Make the prediction
with torch.no_grad():
    output = model(input_data)

# Post-processing
#probabilities = output[0]
probabilities = torch.nn.functional.softmax(output[0], dim=0)
predicted_class = torch.argmax(probabilities).item()




In [ ]:
# Download ImageNet labels
LABELS_URL = 'https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json'
labels = requests.get(LABELS_URL,verify=False).json()


In [ ]:
import json
# Save the list to a JSON file
file_path = 'zoo/imagenet-simple-labels.json'
with open(file_path, 'w') as file:
    json.dump(labels, file)

In [ ]:

# Get the predicted label
predicted_label = labels[predicted_class]

print(f"Predicted class: {predicted_class}")
print(f"Predicted label: {predicted_label}")
print(f"Probability: {probabilities[predicted_class]:.2%}")